In [ ]:
!pip install -U spacy

In [5]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.7 MB/s eta 0:00:00


In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
import spacy
from PyPDF2 import PdfReader
import json
import fitz

In [7]:
nlp = spacy.load('en_core_web_lg')

In [8]:
def extract_text_from_pdf(path):
    # creating a pdf reader object
    reader = PdfReader(path)

    # printing number of pages in pdf file
    print(len(reader.pages))

    # getting a specific page from the pdf file
    text_array = []


    for i in range(len(reader.pages)):
    # extracting text from page
        page = reader.pages[i]
        text = page.extract_text()
        text_array.append(text)

    return text_array

In [9]:
# Process each page using spaCy

def sentence_extraction(filename):

  #get text from pdf
  text_array =  extract_text_from_pdf(filename)
  processed_pages = []
  page_no = 0

  #converting pages into docs for vectorization
  for page in text_array:
      doc = nlp(page)
      processed_pages.append([doc, page_no])
      page_no+=1

  #creating sentences array to access every vectorized sentence
  sentences = []
  for doc, page_no in processed_pages:
      for sentence in doc.sents:
          sentences.append([sentence, page_no])
  return sentences


In [10]:
def highlight_pdf(pdf_path, page_number, sentence):
    # Open the PDF file
    pdf = fitz.open(pdf_path)

    # Select the page based on page_number (starting from 0)
    page = pdf[page_number]

    # Search for the sentence and get its bounding box
    rect = page.search_for(sentence)

    # Add a highlight annotation for the sentence
    for r in rect:
        highlight = page.add_highlight_annot(r)




In [11]:
def create_json_file(context_sents, filename):
  #get the required sentences from the given file
  sentences = sentence_extraction(filename)

  #create a dictionary to dump into the json file
  context_dict = {}
  for input_sent in context_sents:
    context_dict[input_sent] = []

  for input_sent in context_sents:
    # Process the input sentence
    input_doc = nlp(input_sent)

    # Calculate similarity with each sentence
    similarity_scores = []
    for sentence, page_no in sentences:
        similarity_score = input_doc.similarity(sentence)
        similarity_scores.append((sentence, similarity_score, page_no))

    # Sort similarity scores in descending order
    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    i = 0
    # Print the most similar sentences
    for sentence, similarity_score, page_no in similarity_scores:
        temp_dict = {}
        temp_dict['file'] = filename
        temp_dict['page_no'] = page_no
        temp_dict['sentence'] = sentence.text
        temp_dict['similarity_score'] = similarity_score
        context_dict[input_sent].append(temp_dict)
        #highlight_pdf(filename, page_no, sentence.text)  #uncomment to highlight in pdf
        if i==10: #returns only the top 10 sentence matches
          break
        i+=1

    #print(context_dict)

    #write into json file
    json_string = json.dumps(context_dict, indent = 2)
    with open("context_search_results.json", "w") as f:
      f.write(json_string)



In [12]:

#The sentences are of the form list of strings
#followed by the filename as a string
create_json_file(["neural networks are the best for image recognition", "Long term memory in Neural Networks", "IMage segmentation"], '/content/2018_Book_NeuralNetworksAndDeepLearning.pdf')

512


<ipython-input-11-a21d94cb8d3f>:14: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_score = input_doc.similarity(sentence)


{'neural networks are the best for image recognition': [{'file': '/content/2018_Book_NeuralNetworksAndDeepLearning.pdf', 'page_no': 219, 'sentence': 'Convolutional neural networks are used for image\nrecognition and prediction.', 'similarity_score': 0.9343882001078312}, {'file': '/content/2018_Book_NeuralNetworksAndDeepLearning.pdf', 'page_no': 59, 'sentence': '1.6.5 Convolutional Neural Networks\nConvolutional neural networks are biologically inspired networks that are used in computer\nvision for image classiﬁcation and object detection.', 'similarity_score': 0.9264531609446556}, {'file': '/content/2018_Book_NeuralNetworksAndDeepLearning.pdf', 'page_no': 386, 'sentence': 'The two most popular data sets for testing convolutional neural networks are MNIST\nandImageNet .', 'similarity_score': 0.9095713538369533}, {'file': '/content/2018_Book_NeuralNetworksAndDeepLearning.pdf', 'page_no': 72, 'sentence': 'Neural networks are also parameterized models that are learned with continuous opti